## Genrating masks from VIA JSON

In [ ]:
import json
import os
import cv2
import numpy as np


In [ ]:
def generate_masks(json_file, images_dir, masks_dir):
    with open(json_file, 'r') as file:
        data = json.load(file)

    annotations = data['_via_img_metadata'].values()

    for annotation in annotations:
        image_filename = annotation['filename']
        image_path = os.path.join(images_dir, image_filename)
        image = cv2.imread(image_path)

        mask = np.zeros(image.shape[:2], dtype=np.uint8)
        regions = annotation['regions']

        for region in regions:
            shape_attributes = region['shape_attributes']
            region_type = region['region_attributes']['Shape']

            if region_type == 'Circle':
                if shape_attributes['name'] == 'circle':
                    cx = int(shape_attributes['cx'])
                    cy = int(shape_attributes['cy'])
                    radius = int(shape_attributes['r'])
                    cv2.circle(mask, (cx, cy), radius, 255, -1)

            elif region_type == 'Polygon':
                if shape_attributes['name'] == 'polygon':
                    points_x = shape_attributes['all_points_x']
                    points_y = shape_attributes['all_points_y']
                    points = np.array(list(zip(points_x, points_y)), np.int32)
                    points = points.reshape((-1, 1, 2))
                    cv2.fillPoly(mask, [points], 255)

            elif region_type == 'Point':
                if shape_attributes['name'] == 'point':
                    x = int(shape_attributes['cx'])
                    y = int(shape_attributes['cy'])
                    cv2.circle(mask, (x, y), 1, 255, -1)

        mask_filename = os.path.splitext(image_filename)[0] + '.png'
        mask_path = os.path.join(masks_dir, mask_filename)
        cv2.imwrite(mask_path, mask)

In [ ]:
json_file = 'Put the path of the json file here'
images_dir = 'Put the path of the images directory here'
masks_dir = 'Put the path of the masks directory here'

generate_masks(json_file, images_dir, masks_dir)